## Cloud storage python API

> To use the python API, install
```bash
pip install google-cloud-storage
```
> Refer the python samples in cloud storage [documentation](https://cloud.google.com/storage/docs/how-to)

### Authentication

> Reference [document](https://cloud.google.com/docs/authentication/production#providing_credentials_to_your_application)

> Create a **Service account** with **required permissions** and **generate json key**

> Accessing cloud storage from **local machines or VM instances**

```python
    #pip install google-cloud-storage
    from google.cloud import storage

    #pass the service account key as credential
    client = storage.Client.from_service_account_json('test-engine-service.json')
    bucket = client.get_bucket("senesence-bkt-1")
    #destination path in bucket
    blob = bucket.blob("sql/new_from_aravind.txt")
    #upload the new file in local directory
    #Requires Storage writer and Editor permission
    blob.upload_from_filename("new.txt")
```

> Accessing cloud storage from **standard appengine**

>> **appengine** has its **default bucket storage** in google cloud with **5GB** free memory. This bucket has **all permissions** when **accessed from appengine**. 

>> **appengine** accesss the **cloud storage** with **different api**
```bash
pip install GoogleAppEngineCloudStorageClient -t lib
```
this library will work only from **appengine** or **developmentserver**. Therefore, store this library and add the **appengine_config.py**. And **google.appengine** library will be present only in the **appengine environment** or the **development server**. Reference [set up document](https://cloud.google.com/appengine/docs/standard/python/googlecloudstorageclient/setting-up-cloud-storage)

>>simple demonstration with **flask**. Reference [read-write example](https://cloud.google.com/appengine/docs/standard/python/googlecloudstorageclient/read-write-to-cloud-storage) and [api document](https://cloud.google.com/appengine/docs/standard/python/googlecloudstorageclient/functions#listbucket)

```python
        from flask import Flask


        app = Flask(__name__)
        app.debug=True

        @app.route('/')
        def hello():
           #google.appengine will be present only in appengine or development server
           from google.appengine.api import app_identity
           import cloudstorage as gcs

           #GEt default bucket name
           default_bucket_name = app_identity.get_default_gcs_bucket_name()

           out = "Default bucket : " + default_bucket_name

           ## Write to cloud
           filename = '/' + default_bucket_name + '/new.txt'
           gcs_file = gcs.open(filename, 'w', content_type='text/plain')
           gcs_file.write('abcde\n')
           gcs_file.write('Hello!' + '\n')
           gcs_file.close()

           ## Read from cloud
           gcs_file = gcs.open(filename)
           contents = gcs_file.read()
           gcs_file.close()

           out += "<p>Contents :</p>"
           out += "<p>" + contents + "</p>"

           #files CANNOT be appended as the objects are immutable. to append you have to read, modify contents and overwrite 

           #List items 
           blist = gcs.listbucket('/' + default_bucket_name)
           print(list(blist))

           return out 

        if __name__ == '__main__':
            app.run()
```

>> **To test this code**
```bash
 dev_appserver.py app.yaml --default_gcs_bucket_name [DEFAULT_BUCKET_NAME]
```
**default bucket name** can be found under **settings** in **appengine**

>>**app.yaml**

```yaml
        runtime: python27
        api_version: 1
        threadsafe: true

        handlers:
        - url: /.*
          script: main.app

        libraries:
        - name: flask
          version: 0.12
```